In [1]:
CREATE TABLE Sales (
    Id INT NOT NULL IDENTITY(1,1) PRIMARY KEY,
    Product VARCHAR(255),
    Date int,
    Sale int 
);

Commands completed successfully.

Total execution time: 00:00:00.027

In [2]:
INSERT INTO Sales (Product,Date,Sale) VALUES
('Prod1',2017,999999),
('Prod2',2016,99999),
('Prod3',2018,9999);

(3 rows affected)

Total execution time: 00:00:00.019

In [3]:
SELECT * FROM Sales

(3 rows affected)

Total execution time: 00:00:00.061

Id,Product,Date,Sale
1,Prod1,2017,999999
2,Prod2,2016,99999
3,Prod3,2018,9999


In [22]:
SELECT * FROM (
    SELECT 
    [Product],
    [Date],
    [Sale]
    FROM Sales 
) SalesResults
PIVOT (
    SUM([Sale])
    FOR [Date]
    IN (
        [2016],
        [2017],
        [2018],
        [2019]
    )
) AS PivotTable

(3 rows affected)

Total execution time: 00:00:00.010

Product,2016,2017,2018,2019
Prod1,NULL,999999,NULL,NULL
Prod2,99999,NULL,NULL,NULL
Prod3,NULL,NULL,9999,NULL


In [23]:
CREATE PROCEDURE dbo.DynamicPivotTableInSq
  @ColumnToPivot  NVARCHAR(255),
  @ListToPivot    NVARCHAR(255)
AS
BEGIN
 
  DECLARE @SqlStatement NVARCHAR(MAX)
  SET @SqlStatement = N'
    SELECT * FROM (
    SELECT 
    [Product],
    [Date],
    [Sale]
    FROM Sales 
) SalesResults
PIVOT (
    SUM([Sale])
    FOR ['+@ColumnToPivot+']
    IN (
        '+@ListToPivot+'
    )
) AS PivotTable
  ';
 
  EXEC(@SqlStatement)
 
END

Commands completed successfully.

Total execution time: 00:00:00.004

In [24]:
EXEC dbo.DynamicPivotTableInSq
  N'Date'
  ,N'[2016],[2017],[2018]'

(3 rows affected)

Total execution time: 00:00:00.007

Product,2016,2017,2018
Prod1,NULL,999999,NULL
Prod2,99999,NULL,NULL
Prod3,NULL,NULL,9999


In [3]:
DECLARE @cols AS NVARCHAR(MAX),
    @query  AS NVARCHAR(MAX)

select @cols = STUFF((SELECT ',' + QUOTENAME(Date) 
                    from Sales
                    group by Date
                    order by Date
            FOR XML PATH(''), TYPE
            ).value('.', 'NVARCHAR(MAX)') 
        ,1,1,'')

set @query = 'CREATE VIEW  [mv_view1 ] AS 
SELECT Product,' + @cols + ' from 
             (
                select Product, Date, Sale
                from Sales
            ) x
            pivot 
            (
                sum(Sale)
                for Date in (' + @cols + ')
            ) p '

execute(@query);

: Msg 2714, Level 16, State 3, Procedure mv_view1 , Line 1
There is already an object named 'mv_view1 ' in the database.

Total execution time: 00:00:00.027

In [4]:
SELECT * FROM mv_view1 

(3 rows affected)

Total execution time: 00:00:00.128

Product,2016,2017,2018
Prod1,NULL,999999,NULL
Prod2,99999,NULL,NULL
Prod3,NULL,NULL,9999


In [4]:
CREATE TABLE Employee (
    Id INT NOT NULL IDENTITY(1,1) PRIMARY KEY,
    Name VARCHAR(255),
    Manager_id int 
);

Commands completed successfully.

Total execution time: 00:00:00.011

In [5]:
INSERT INTO Employee (Name,Manager_id)
VALUES 
('Silva',NULL),
('Perera',1),
('Sandun',1),
('Mathan',2),
('Abdul',2),
('Mihindu',3);

(6 rows affected)

Total execution time: 00:00:00.006

In [6]:
SELECT * FROM Employee;

(6 rows affected)

Total execution time: 00:00:00.006

Id,Name,Manager_id
1,Silva,NULL
2,Perera,1
3,Sandun,1
4,Mathan,2
5,Abdul,2
6,Mihindu,3


In [7]:
SELECT 
sub.Name AS Employee_Name,
sup.Name AS Manager_Name
FROM Employee sub
JOIN Employee sup
ON sub.Manager_id = sup.Id;

(5 rows affected)

Total execution time: 00:00:00.013

Employee_Name,Manager_Name
Perera,Silva
Sandun,Silva
Mathan,Perera
Abdul,Perera
Mihindu,Sandun


In [31]:
CREATE TABLE Product (
    Id INT NOT NULL IDENTITY(1,1) PRIMARY KEY,
    Product VARCHAR(255),
    ProductType VARCHAR(255),
    ProductCat VARCHAR(255) 
);

Commands completed successfully.

Total execution time: 00:00:00.008

In [32]:
INSERT INTO Product (Product,ProductType,ProductCat) VALUES
('Prod1','ProdType1','ProdCat1'),
('Prod2','ProdType2','ProdCat2'),
('Prod3','ProdType3','ProdCat3');

(3 rows affected)

Total execution time: 00:00:00.006

In [33]:
SELECT * FROM Product

(3 rows affected)

Total execution time: 00:00:00.004

Id,Product,ProductType,ProductCat
1,Prod1,ProdType1,ProdCat1
2,Prod2,ProdType2,ProdCat2
3,Prod3,ProdType3,ProdCat3


In [39]:
SELECT 
prod.Product, 
prod.ProductType, 
prod.ProductCat, 
sales.Date, 
sales.Sale
FROM Product as prod
JOIN Sales as sales
ON prod.Product = sales.Product;

(3 rows affected)

Total execution time: 00:00:00.005

Product,ProductType,ProductCat,Date,Sale
Prod1,ProdType1,ProdCat1,2017,999999
Prod2,ProdType2,ProdCat2,2016,99999
Prod3,ProdType3,ProdCat3,2018,9999


In [8]:
CREATE TABLE SalesSummarry (
    Id INT NOT NULL IDENTITY(1,1) PRIMARY KEY,
    Product VARCHAR(255),
    Product_Type VARCHAR(255),
    Product_Category VARCHAR(255),
    Sales int 
);



Commands completed successfully.

Total execution time: 00:00:00.012

In [9]:
INSERT INTO SalesSummarry (Product,Product_Type,Product_Category,Sales) VALUES
('Tshirt','Clothing','Top',4500),
('Jeans','Clothing','Bottom',3000),
('Shirt','Clothing','Top',7500),
('Shorts','Clothing','Bottom',5000),
('Bracelet','Jewelery','handJewels',1000),
('Bangels','Jewelery','handJewels',1200),
('diamondRings','Jewelery','rings',1100),
('goldRings','Jewelery','rings',200);

(8 rows affected)

Total execution time: 00:00:00.048

In [10]:
SELECT * FROM SalesSummarry;

(8 rows affected)

Total execution time: 00:00:00.009

Id,Product,Product_Type,Product_Category,Sales
1,Tshirt,Clothing,Top,4500
2,Jeans,Clothing,Bottom,3000
3,Shirt,Clothing,Top,7500
4,Shorts,Clothing,Bottom,5000
5,Bracelet,Jewelery,handJewels,1000
6,Bangels,Jewelery,handJewels,1200
7,diamondRings,Jewelery,rings,1100
8,goldRings,Jewelery,rings,200


In [11]:
SELECT  Product_Type, Product_Category,Product, SUM(SalesSummarry.Sales) as TotalSales
FROM SalesSummarry
GROUP BY ROLLUP(Product_Type, Product_Category , Product)

(15 rows affected)

Total execution time: 00:00:00.028

Product_Type,Product_Category,Product,TotalSales
Clothing,Bottom,Jeans,3000
Clothing,Bottom,Shorts,5000
Clothing,Bottom,NULL,8000
Clothing,Top,Shirt,7500
Clothing,Top,Tshirt,4500
Clothing,Top,NULL,12000
Clothing,NULL,NULL,20000
Jewelery,handJewels,Bangels,1200
Jewelery,handJewels,Bracelet,1000
Jewelery,handJewels,NULL,2200


In [12]:
SELECT  Product_Type, Product_Category,Product, SUM(SalesSummarry.Sales) as TotalSales
FROM SalesSummarry
GROUP BY CUBE(Product_Type,Product_Category, Product)

(43 rows affected)

Total execution time: 00:00:00.013

Product_Type,Product_Category,Product,TotalSales
Jewelery,handJewels,Bangels,1200
NULL,handJewels,Bangels,1200
NULL,NULL,Bangels,1200
Jewelery,handJewels,Bracelet,1000
NULL,handJewels,Bracelet,1000
NULL,NULL,Bracelet,1000
Jewelery,rings,diamondRings,1100
NULL,rings,diamondRings,1100
NULL,NULL,diamondRings,1100
Jewelery,rings,goldRings,200


In [ ]:

CREATE TABLE sales  
( product varchar2(50) NOT NULL,  
  date_year number(10),
  sale number(10)
); 


INSERT INTO sales  
(product, date_year, sale)  
VALUES  
('Prod1',2017,999999);  

INSERT INTO sales  
(product, date_year, sale)  
VALUES  
('Prod2',2016,99999);  

INSERT INTO sales  
(product, date_year, sale)  
VALUES  
('Prod3',2018,9999);  

INSERT INTO sales  
(product, date_year, sale)  
VALUES  
('Prod3',2016,999); 

INSERT INTO sales  
(product, date_year, sale)  
VALUES  
('Prod4',2019,99);

select * from sales;

declare
 sqlqry clob;
 cols clob;

begin
  select listagg('''' || date_year || ''' as "' || date_year || '"', ',') within group (order by date_year)
  into   cols
  from   (select distinct date_year from sales);


  sqlqry :=
  'create materialized view mv 
refresh complete on demand
as   
  select * from
  (
      select *
      from sales
  )
  pivot
  (
    MIN(sale) for date_year in (' || cols  || ')
  )';

  execute immediate sqlqry;
end;

select * from mv;

-- select * from cols;

-- DROP MATERIALIZED VIEW mv;
-- DROP TABLE sales PURGE;


-- DROP MATERIALIZED VIEW mv_view1;
-- DROP TABLE sales PURGE;



In [ ]:
CREATE TABLE sales_range 
(s_productid  NUMBER,
   s_saledate   DATE,
   s_custid     NUMBER,
   s_totalprice NUMBER,
   s_country VARCHAR2(30))
COMPRESS
PARTITION BY RANGE(s_saledate)
(PARTITION sales_jan2000 VALUES LESS THAN(TO_DATE('02/01/2000','DD/MM/YYYY')),
 PARTITION sales_feb2000 VALUES LESS THAN(TO_DATE('03/01/2000','DD/MM/YYYY')),
 PARTITION sales_mar2000 VALUES LESS THAN(TO_DATE('04/01/2000','DD/MM/YYYY')),
 PARTITION sales_apr2000 VALUES LESS THAN(TO_DATE('05/01/2000','DD/MM/YYYY')));
 
 
 CREATE TABLE sales_list
(s_productid  NUMBER,
   s_saledate   DATE,
   s_custid     NUMBER,
   s_totalprice NUMBER,
   s_country VARCHAR2(30))
PARTITION BY LIST(s_country)
(PARTITION sales_west VALUES('America', 'Canada') COMPRESS,
 PARTITION sales_east VALUES('Japan', 'China', 'Korea'),
 PARTITION sales_central VALUES('India', 'SriLanka'));
 
 
 CREATE TABLE quarterly_regional_sales
(s_productid  NUMBER,
   s_saledate   DATE,
   s_custid     NUMBER,
   s_totalprice NUMBER,
   s_country VARCHAR2(30))
PARTITION BY RANGE (s_saledate)
SUBPARTITION BY LIST (s_country)
(
PARTITION q1_1999 VALUES LESS THAN(TO_DATE('1-APR-1999','DD-MON-YYYY'))
(SUBPARTITION q1_1999_sales_west VALUES('America', 'Canada'), 
SUBPARTITION q1_1999_sales_east VALUES('Japan', 'China', 'Korea'),
SUBPARTITION q1_1999_sales_central VALUES('India', 'SriLanka')));



CREATE TABLE sales_list_country
( s_productid  VARCHAR2(30),
   s_saledate   DATE,
   s_custid     NUMBER,
   s_totalprice NUMBER,
   s_productcategory VARCHAR2(30),
   s_country VARCHAR2(30)
) PARTITION BY LIST (s_country)
SUBPARTITION BY LIST (s_productcategory)
SUBPARTITION TEMPLATE
( SUBPARTITION perishable VALUES ('DAIRY','PRODUCE','MEAT','BREAD')
, SUBPARTITION non_perishable VALUES ('CANNED','PACKAGED')
, SUBPARTITION durable VALUES ('TOYS','KITCHENWARE')
)
( PARTITION sales_west VALUES('America', 'Canada') ,
 PARTITION sales_east VALUES('Japan', 'China', 'Korea'),
 PARTITION sales_central VALUES('India', 'SriLanka')
);

CREATE INDEX s_country_ix ON sales_list_country(s_country)
LOCAL PARALLEL NOLOGGING;

CREATE INDEX s_productid_ix ON sales_list_country(s_productid)
LOCAL PARALLEL NOLOGGING;